In [1]:
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
from pymilvus import connections, Collection
import pandas as pd
from underthesea import ner
import re
import unicodedata

# Kết nối Milvus

In [3]:
connections.connect("default", host="localhost", port="19530")
collection = Collection("hotels_collection_mpnet_base_v2")

# SBERT - BM25

In [4]:
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

In [5]:
def detect_city(query):
    cities = {
        "hồ chí minh": "Hồ Chí Minh",
        "tp hồ chí minh": "Hồ Chí Minh",
        "tp.hcm": "Hồ Chí Minh",
        "sài gòn": "Hồ Chí Minh",
        "hà nội": "Hà Nội",
        "đà nẵng": "Đà Nẵng",
        "phú quốc": "Phú Quốc",
        "nha trang": "Nha Trang",
        "hội an": "Hội An",
        "đà lạt": "Đà Lạt",
        "sa pa": "Sa Pa",
        "sapa": "Sa Pa",
        "huế": "Huế",
        "vũng tàu": "Vũng Tàu"
    }
    query_lower = query.lower()
    
    # Rule-based
    for k, v in cities.items():
        if k in query_lower:
            return v
    
    # NER-based
    for word, _, _, tag in ner(query):
        if tag.endswith("LOC"):
            if word.title() in df["Location"].unique():
                print(f'Địa danh nhận dạng: {word}')
                return word.title()
    
    return None

def clean_text_for_query(text):
    if not isinstance(text, str):
        return ""

    text = unicodedata.normalize('NFC', text)
    text = text.lower()

    text = re.sub(r"[^\w\s/\-]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()

    return text

In [6]:
def prepare_query(query):
    city = detect_city(query)
    tokenized_query = query.lower().split()
    expr = ""
    bm25_scores = []
    
    if city:
        expr = f'Location like "%{city}%"'
        
        results = collection.query(
            expr=expr,
            output_fields=["HotelID", "Description"]
        )

        if results:
            filtered_docs = [r['Description'] for r in results]
            tokenized_filtered = [d.lower().split() for d in filtered_docs]
            bm25_local = BM25Okapi(tokenized_filtered)
            bm25_scores = bm25_local.get_scores(tokenized_query)

             # Normalize an toàn
            if bm25_scores.max() != bm25_scores.min():
                bm25_scores = (bm25_scores - bm25_scores.min()) / (bm25_scores.max() - bm25_scores.min())
            else:
                bm25_scores = [0] * len(bm25_scores)
        
    semantic_query = query.lower().replace(city.lower(), "").strip() if city else query
    return semantic_query, expr, bm25_scores

In [22]:
def hybrid_search(query, alpha=1, top_k=20):
    
    semantic_query, expr, bm25_scores = prepare_query(query)
    semantic_query = clean_text_for_query(semantic_query)
    
    query_emb = model.encode([semantic_query], normalize_embeddings=True)
    search_params = {"metric_type": "COSINE", "params": {"M": 8, "efConstruction": 64}}

    results = collection.search(
        data=query_emb,
        anns_field="TextForEmbedding",
        param=search_params,
        limit=top_k,
        expr=expr,
        output_fields=["HotelID", "Description", "NameHotel"]
    )

    milvus_hits = []
    for hits in results:
        for idx, hit in enumerate(hits):
            bm25_score = bm25_scores[idx] if idx < len(bm25_scores) else 0
            milvus_hits.append({
                "HotelID": hit.entity.get("HotelID"),
                "Name Hotel": hit.entity.get("NameHotel"),
                "Descriptions": hit.entity.get("Description"),
                "semantic_score": hit.distance,
                "bm25_score": bm25_score
            })
            

    for h in milvus_hits:
        h["final_score"] = alpha * h["semantic_score"] + (1 - alpha) * h["bm25_score"]

    # Sắp xếp theo final score
    milvus_hits = sorted(milvus_hits, key=lambda x: x["final_score"], reverse=True)[:top_k]
        
    ids = [str(h['HotelID']) for h in milvus_hits]
    print("📋 HotelID list:",",".join(ids))
    print("=" * 80)

    # In kết quả
    for h in milvus_hits:
        print(f"🏨 {h['Name Hotel']} (HotelId: {h['HotelID']}) (Score: {h['final_score']:.4f}) (Semantic: {h['semantic_score']:.4f}) (BM25: {h['bm25_score']:.4f})")
        print(f"   {h['Descriptions']}\n")

# Test

In [23]:
# query = "Resort lãng mạn cho cặp đôi ở Đà Nẵng"
# query = "Resort gần biển yên tĩnh"
# query = "Khách sạn cho gia đình có trẻ em"
query ="khách sạn gần biển đà nẵng"
hybrid_search(query)

📋 HotelID list: 342,1074,3496,3937,2969,570,549,3038,131,1151,474,1062,2735,1222,2559,1494,1362,1059,276,129
🏨 CODI SEA Hotel & Travel (HotelId: 342) (Score: 0.8688) (Semantic: 0.8688) (BM25: 0.3825)
   Tọa lạc tại thành phố Đà Nẵng, cách Bãi biển Mỹ Khê 300 m, CODI SEA Hotel & Travel có dịch vụ tiền sảnh, phòng nghỉ không gây dị ứng, sảnh khách chung, WiFi miễn phí trong toàn bộ khuôn viên và khu vườn. Khách sạn 2 sao này có dịch vụ phòng và bàn đặt tour. Nơi đây cung cấp dịch vụ lễ tân 24 giờ, bếp chung và dịch vụ thu đổi ngoại tệ cho khách. Tất cả phòng nghỉ tại CODI SEA Hotel & Travel rooms đều được trang bị máy điều hòa, TV truyền hình vệ tinh màn hình phẳng, tủ lạnh, ấm đun nước, vòi xịt/chậu rửa vệ sinh, dép, bàn làm việc, phòng tắm riêng và tủ để quần áo. Một số phòng có sân trong và tầm nhìn ra quang cảnh biển. Các phòng đi kèm ga trải giường và khăn tắm. Chỗ nghỉ phục vụ bữa sáng à la carte hoặc bữa sáng kiểu Á. CODI SEA Hotel & Travel có sân hiên. Đi xe đạp là hoạt động được